### Import Dependencies

In [1]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import cv2

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu,True)

In [3]:
gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

## Draw Keypoints

In [4]:
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    
    for kp in shaped:
        ky,kx , kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx),int(ky)),4,(0,255,0),-1)

## Draw Edges

In [5]:
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [6]:
def draw_connections(frame,keypoints,edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1,p2 = edge
        y1,x1,c1 = shaped[p1]
        y2,x2,c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2>confidence_threshold):
            cv2.line(frame, (int(x1),int(y1)),(int(x2),int(y2)),(255,0,0),2)

### Load Model

In [7]:
interpreter = tf.lite.Interpreter(model_path='lite-model_movenet_singlepose_thunder_3.tflite')
interpreter.allocate_tensors()

In [9]:
target_width = 256
target_height = 256

cap = cv2.VideoCapture(0)

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

last_position = None
last_time = None

maxVelocity_x = 0
maxVelocity_y = 0

while cap.isOpened():
    ret,frame = cap.read()
    frame_resized = cv2.resize(frame, (target_width, target_height))
    
    # Reshape Image
    img = frame_resized.copy()
    img = tf.image.resize_with_pad(np.expand_dims(img,axis=0),256,256)
#     img = tf.image.resize_with_pad(tf.expand_dims(img,axis=0),192,256)
    input_image = tf.cast(img,dtype=tf.float32)
    
    # Setup input and output
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    # Make predictions
    interpreter.set_tensor(input_details[0]['index'],np.array(input_image))
    interpreter.invoke()
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
    keypoints = keypoints_with_scores.reshape((-1, 3))[:, :2]
#     print(keypoints_with_scores)
    
    
    # Velocity
    left_wrist_x, left_wrist_y = keypoints[9][1], keypoints[9][0]
    if last_position is None:
            last_position = [left_wrist_x, left_wrist_y]
            last_time = cv2.getTickCount() / cv2.getTickFrequency()
            continue

    # Calculate the time interval since the last frame
    current_time = cv2.getTickCount() / cv2.getTickFrequency()
    time_interval = current_time - last_time
    last_time = current_time

    # Calculate the velocity of the person's nose
    position = (left_wrist_x, left_wrist_y)
    delta_x = position[0] - last_position[0]
    delta_y = position[1] - last_position[1]
    last_position = position
    print(last_position)
    velocity_x = delta_x / time_interval
    velocity_y = delta_y / time_interval
    
    maxVelocity_x = max(maxVelocity_x,abs(velocity_x))
    maxVelocity_y = max(maxVelocity_y,abs(velocity_y))
    
    print("Velocity X: ",velocity_x)
    print("Velocity y: ",velocity_y)
    
    
    
    # cv2.putText(frame, f'Max Speed: ({maxVelocity_x:.2f}, {maxVelocity_y:.2f})', (10, 30),
    #                 cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    # cv2.putText(frame, f'Velocity: ({velocity_x:.2f}, {velocity_y:.2f})', (10, 60),
    #                 cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
#     cv2.imshow('frame', frame)
    
    
    draw_connections(frame,keypoints_with_scores,EDGES, 0.4)
    draw_keypoints(frame,keypoints_with_scores, 0.4)
    
    
    cv2.imshow('Movenet Lighting',frame)
    
    if cv2.waitKey(10) & 0xFF==ord('q'):
        break
        
        
cap.release()
cv2.destroyAllWindows()

(0.94034445, 0.7409552)
Velocity X:  0.043902319872212235
Velocity y:  -0.08979477422677033
(0.9287611, 0.72762835)
Velocity X:  -0.1011262846382873
Velocity y:  -0.1163475763801907
(0.9330026, 0.7421926)
Velocity X:  0.04300599769731497
Velocity y:  0.1476732648305722
(0.9375003, 0.74239683)
Velocity X:  0.041499953770688415
Velocity y:  0.0018841866658500445
(0.90440273, 0.7086101)
Velocity X:  -0.3680337012070625
Velocity y:  -0.37569680331382943
(0.90220666, 0.7040359)
Velocity X:  -0.023232063537587732
Velocity y:  -0.048390463903623254
(0.93624145, 0.7414006)
Velocity X:  0.3682071774793749
Velocity y:  0.4042322320481124
(0.9357209, 0.7438878)
Velocity X:  -0.0056783791444713645
Velocity y:  0.027132417833562475
(0.9008843, 0.7104899)
Velocity X:  -0.3586017088748913
Velocity y:  -0.3437910169042739
(0.90184015, 0.7114408)
Velocity X:  0.010706357848993427
Velocity y:  0.010650279152272379
(0.8965099, 0.70864403)
Velocity X:  -0.055974945556791146
Velocity y:  -0.029369836798905

In [49]:
frame.shape

(480, 640, 3)